In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

INPUT_DIR = '/Users/fuchunhsieh/Desktop/Fall_2023/Comp_Sci_539/Project/free-spoken-digit-dataset-master/recordings'
OUTPUT_DIR = '/Users/fuchunhsieh/Desktop/Fall_2023/Comp_Sci_539/Project'

# list all the files and directories within a specified directory
parent_list = os.listdir(INPUT_DIR)
print(len(parent_list)) # 6 speakers, 10 digits, 50 recordings for each digit
for i in range(10):
    print(parent_list[i])

3000
5_nicolas_9.wav
3_yweweler_14.wav
1_lucas_5.wav
2_george_44.wav
4_yweweler_38.wav
3_yweweler_28.wav
4_yweweler_10.wav
3_george_1.wav
1_nicolas_15.wav
1_jackson_42.wav


In [2]:
# for i in range(5):
#     # wave.open open a .wav file
#     signal_wave  = wave.open(os.path.join(INPUT_DIR, parent_list[i]), 'r')
#     # sample rate: the number of audio sampls taken per second
#     # setting a low num_frames can result in shorter audio clip
#     # set to a higher value will result in read all that are available and remains fill with zero or silence
#     # to get frames, here single_wave.fetnframes()
#     sample_rate = 16000
#     # print(signal_wave.readframes(sample_rate))
#     sig = np.frombuffer(signal_wave.readframes(sample_rate), dtype=np.int16)
#     print(sig.shape)
#     plt.figure(figsize=(8,4))
#     plot_a = plt.subplot(121)
#     plot_a.set_title(parent_list[i])
#     plot_a.plot(sig)
#     plot_a.set_xlabel('sample rate * time')
#     plot_a.set_ylabel('energy')

#     plot_b = plt.subplot(122)
#     plot_b.specgram(sig, NFFT=1024, Fs=sample_rate, noverlap=900)
#     plot_b.set_xlabel('Time')
#     plot_b.set_ylabel('Frequency')

# plt.show()

In [3]:
# def get_wav_info(wav_file):
#     wav = wave.open(wav_file, 'r')
#     frames = wav.readframes(-1)
#     sound_info = np.frombuffer(frames, 'int16')
#     frame_rate = wav.getframerate()
#     wav.close()
#     return sound_info, frame_rate
# if not os.path.exists(os.path.join(OUTPUT_DIR, 'audio-images')):
#     os.mkdir(os.path.join(OUTPUT_DIR, 'audio-images'))
#     for filename in os.listdir(INPUT_DIR):
#         if "wav" in filename:
#             file_path = os.path.join(INPUT_DIR, filename)
#             file_stem = Path(file_path).stem
#             target_dir = f'class_{file_stem[0]}'
#             dist_dir = os.path.join(OUTPUT_DIR, 'audio-images', target_dir)
#             file_dist_path = os.path.join(dist_dir, file_stem)
#             if not os.path.exists(dist_dir):
#                 os.makedirs(dist_dir)
#             if not os.path.exists(file_dist_path + '.png'):
#                 file_stem = Path(file_path).stem
#                 sound_info, frame_rate = get_wav_info(file_path)
#                 pylab.specgram(sound_info, Fs = frame_rate)
#                 pylab.savefig(f'{file_dist_path}.png')
#                 pylab.close()
# path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
# print("Classes: \n")
# for i in range(10):
#     print(path_list[i])

# path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_1'))
# print("\nA few example files: \n")
# for i in range(10):
#     print(path_list[i])

In [4]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
BATCH_SIZE = 32
N_CHANNELS = 1
N_CLASSES = 10

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    batch_size = BATCH_SIZE,
    validation_split = 0.2,
    directory = os.path.join(OUTPUT_DIR, 'audio-images'),
    shuffle = True,
    color_mode = 'grayscale',
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    subset = "training",
    seed = 42
)

valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    batch_size = BATCH_SIZE,
    validation_split = 0.2,
    directory = os.path.join(OUTPUT_DIR, 'audio-images'),
    shuffle = True,
    color_mode = 'grayscale',
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    subset = "validation",
    seed = 42
)

Found 3000 files belonging to 10 classes.
Using 2400 files for training.
Found 3000 files belonging to 10 classes.
Using 600 files for validation.


In [5]:
# plt.figure(figsize=(12,12))
# for images, labels in train_dataset.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i+1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")
# plt.show()

In [6]:
# def prepare(ds, augment=False):
#     rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
#     flip_and_rotate = tf.keras.Sequential(
#         [tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
#          tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)])
#     ds = ds.map(lambda x, y: (rescale(x, training=True), y))
#     if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
#     return ds
# train_dataset = prepare(train_dataset, augment=False)
# valid_dataset = prepare(valid_dataset, augment=False)

In [7]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                        

In [8]:
history = model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
75/75 [==============================] - 3s 36ms/step - loss: 1.2843 - accuracy: 0.6021 - val_loss: 3.5396 - val_accuracy: 0.2117
Epoch 2/10
75/75 [==============================] - 3s 35ms/step - loss: 0.4044 - accuracy: 0.8733 - val_loss: 1.2565 - val_accuracy: 0.6133
Epoch 3/10
75/75 [==============================] - 3s 36ms/step - loss: 0.2347 - accuracy: 0.9371 - val_loss: 0.5752 - val_accuracy: 0.8100
Epoch 4/10
75/75 [==============================] - 3s 35ms/step - loss: 0.1671 - accuracy: 0.9575 - val_loss: 0.2974 - val_accuracy: 0.8833
Epoch 5/10
75/75 [==============================] - 3s 37ms/step - loss: 0.1181 - accuracy: 0.9708 - val_loss: 0.3619 - val_accuracy: 0.8783
Epoch 6/10
75/75 [==============================] - 3s 39ms/step - loss: 0.1003 - accuracy: 0.9750 - val_loss: 0.2218 - val_accuracy: 0.9350
Epoch 7/10
75/75 [==============================] - 3s 36ms/step - loss: 0.0776 - accuracy: 0.9792 - val_loss: 0.2930 - val_accuracy: 0.9217
Epoch 8/10
75

In [9]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
# model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# # Compile model
# model.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer=tf.keras.optimizers.RMSprop(),
#     metrics=['accuracy'],
# )

# # Train model for 10 epochs, capture the history

In [10]:
# # Plot the loss curves for training and validation.
# history_dict = history.history
# loss_values = history_dict['loss']
# val_loss_values = history_dict['val_loss']
# epochs = range(1, len(loss_values)+1)

# plt.figure(figsize=(8,6))
# plt.plot(epochs, loss_values, 'bo', label='Training loss')
# plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [11]:
# # Plot the accuracy curves for training and validation.
# acc_values = history_dict['accuracy']
# val_acc_values = history_dict['val_accuracy']
# epochs = range(1, len(acc_values)+1)

# plt.figure(figsize=(8,6))
# plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
# plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

In [12]:
# # Compute the final loss and accuracy
# final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
# print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                        

In [14]:
import time

# Start the timer.
t0 = time.process_time()

final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)

# Stop the timer.0
t1 = time.process_time()

print((t1-t0), len(valid_dataset), 'batches')
avg_time = (t1 - t0) / len(valid_dataset)
print("Average time per batch: ", avg_time)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

# penalty = ... # whatever, we'll find this out later.
# adjusted_acc = final_acc / (1 + penalty * avg_time)

# print("Time adjusted final accuracy: {1:.6f}".format(adjusted_acc))

3.900548000000015 19 batches
Average time per batch:  0.20529200000000078
Final loss: 0.172011, final accuracy: 0.950000
